In [ ]:
# !conda install pytorch:pytorch

Channels:
 - defaults
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /usr/local/Caskroom/miniconda/base/envs/bitnet-cpp

  added / updated specs:
    - pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    deprecated-1.2.13          |   py39hecd8cb5_0          16 KB
    filelock-3.17.0            |   py39hecd8cb5_0          33 KB
    fsspec-2025.7.0            |   py39he13d532_0         542 KB
    gmp-6.3.0                  |       h6d0c2b6_0         581 KB
    gmpy2-2.2.1                |   py39h2cec913_0         214 KB
    jinja2-3.1.6               |   py39hecd8cb5_0         280 KB
    libabseil-20250127.0       | cxx17_h548131f_0         1.2 MB
    libprotobuf-5.29.3         |       h66a43dd_1         2.8 MB
    libtorch-2.6.0             |cpu_openblas_h4faba3d_6        48.6 MB
    libuv-1.48.0               |       h46256e1

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pennylane as qml

ImportError: dlopen(/usr/local/Caskroom/miniconda/base/envs/bitnet-cpp/lib/python3.9/site-packages/torch/_C.cpython-39-darwin.so, 0x0002): Symbol not found: __ZN2at3cpu19is_cpu_support_vnniEv
  Referenced from: <6CB41E0C-65D2-39DB-9F32-D8537790DD07> /usr/local/Caskroom/miniconda/base/envs/bitnet-cpp/lib/python3.9/site-packages/torch/lib/libtorch_python.dylib
  Expected in:     <078DF3B7-D63C-3CEF-94C1-C063C5E5A468> /usr/local/Caskroom/miniconda/base/envs/bitnet-cpp/lib/libtorch_cpu.dylib

In [ ]:
# Load CIFAR-10 sample
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
images, labels = next(iter(trainloader))

In [ ]:
# Define quantum device
dev = qml.device("braket.aws.qubit", 
                 device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1", 
                 wires=4)

In [ ]:
# Quantum circuit
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    for i in range(4):
        qml.RY(inputs[i], wires=i)
    for i in range(4):
        qml.RZ(weights[i], wires=i)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])
    for i in range(4):
        qml.RY(weights[i+4], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [ ]:
# Hybrid CNN + Quantum model
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 4)
        self.quantum_layer = qml.qnn.TorchLayer(quantum_circuit, {"weights": (8,)})
        self.fc2 = nn.Linear(4, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.tanh(self.fc1(x))
        x = self.quantum_layer(x)
        x = self.fc2(x)
        return x

In [ ]:
# Forward pass
model = HybridModel()
output = model(images)

print(f"Input shape: {images.shape}")
print(f"Output shape: {output.shape}")
print(f"Output:\n{output}")